In [ ]:
!git clone https://github.com/GATECH-EIC/TinyML-Contest-Solution.git

Cloning into 'TinyML-Contest-Solution'...
remote: Enumerating objects: 30547, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 30547 (delta 0), reused 2 (delta 0), pack-reused 30545
Receiving objects: 100% (30547/30547), 153.75 MiB | 11.99 MiB/s, done.
Resolving deltas: 100% (84/84), done.
Updating files: 100% (30513/30513), done.


In [ ]:
%cd TinyML-Contest-Solution/Training

/content/TinyML-Contest-Solution/Training


In [ ]:
# -*- coding: utf-8 -*-
"""arritmia_cardiaca.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1kIePtG_yixpzcbjVl5mY7nwQkAytTe_V
"""

# Credits to https://towardsdatascience.com/multi-task-learning-for-computer-vision-classification-with-keras-36c52e6243d2

# Imports
print("start imports")
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import time
import os
import csv
import random
from tensorflow import keras
print("end imports")


# Custom stuff from https://github.com/GATECH-EIC/TinyML-Contest-Solution/blob/master/Training/select_model.py
#from helpers.cosine_annealing import CosineAnnealingScheduler
from swa.tfkeras import SWA


""" Functions for loading data """
def txt_to_numpy(filename, row=1250):
    file = open(filename)
    lines = file.readlines()
    datamat = np.arange(row, dtype=np.float)
    row_count = 0
    for line in lines:
        line = line.strip().split(' ')
        datamat[row_count] = line[0]
        row_count += 1

    return datamat

"""
AFb,Atrial Fibrillation
AFt,Atrial Flutter
SR,Sinus Rhythm
SVT,Supraventricular Tachycardia
VFb,Ventricular Fibrillation
VFt,Ventricular Flutter
VPD,Ventricular Premature Depolarizations
VT,Ventricular Tachycardia
"""
def txt_to_disease_type(txt):
    txt = txt.upper()
    if txt == "AFB":
      return [1, 0, 0, 0, 0, 0, 0, 0]
    if txt == "AFT":
      return [0, 1, 0, 0, 0, 0, 0, 0]
    if txt == "SR":
      return [0, 0, 1, 0, 0, 0, 0, 0]
    if txt == "SVT":
      return [0, 0, 0, 1, 0, 0, 0, 0]
    if txt == "VFB":
      return [0, 0, 0, 0, 1, 0, 0, 0]
    if txt == "VFT":
      return [0, 0, 0, 0, 0, 1, 0, 0]
    if txt == "VPD":
      return [0, 0, 0, 0, 0, 0, 1, 0]
    if txt == "VT":
      return [0, 0, 0, 0, 0, 0, 0, 1]
    raise "A string was not recognized as a valid input on txt_to_disease_type func!"

"""
def read_data(csv_path, imgs_folder="./tinyml_contest_data_training"):
    x, y1, y2 = [], [], []
    with open(csv_path, "r") as csv_file:
      reader = csv.reader(csv_file)
      next(reader)
      for item in reader: # item[0] = label; item[1] = filename
        x.append(txt_to_numpy(os.path.join(imgs_folder, item[1]), 1250))
        y1.append([1, 0] if int(item[0]) == 0 else [0, 1]) # Label for life threat
        y2.append(txt_to_disease_type(item[1].split("-")[1]))
    x, y1, y2 = sklearn.utils.shuffle(x, y1, y2)
    return np.array(x), np.array(y1), np.array(y2)
"""
def read_data(csv_path, imgs_folder="./tinyml_contest_data_training", augmentation=False, flip_peak=True, flip_time=True, add_noise=True): #, flip_peak=False, flip_time=False, add_noise=False):
    x, y1, y2, y3, y4, y5 = [], [], [], [], [], []
    print("passed here")
    with open(csv_path, "r") as csv_file:
      reader = csv.reader(csv_file)
      next(reader)
      print("Reading data...")
      counter = 0
      for item in reader: # item[0] = label; item[1] = filename
        x.append(txt_to_numpy(os.path.join(imgs_folder, item[1]), 1250))
        y1.append([1, 0] if int(item[0]) == 0 else [0, 1]) # Label for life threat
        y2.append(txt_to_disease_type(item[1].split("-")[1]))
        counter +=1
        if counter % 200 == 0:
          print(f"{counter} entries read...")
    x, y1, y2 = sklearn.utils.shuffle(x, y1, y2)
    x = np.array(x)

    # y3 creation
    for sample in x:
       y3.append(np.mean(sample))
       stdeviation = np.std(sample)
       y4.append(stdeviation)
       y5.append(stdeviation ** 2)

    if augmentation:
      #x = x + np.random.normal(0, random.random()*np.amax(x)*0.05, (len(x), 1))

      for i in range(len(x)):
        flip_p = random.random()
        flip_t = random.random()
        if flip_p < 0.5 and flip_peak:
          x[i] = -x[i]
        if flip_t < 0.5 and flip_time:
          x[i] = np.flip(x[i])
        if add_noise:
          print(f"i={i}")
          max_peak = x[i].max() * 0.05
          factor = random.random()
        # factor = 1
          noise = np.random.normal(0, factor * max_peak, x[0].shape)
          x[i] = x[i] + noise
    return x, np.array(y1), np.array(y2), np.array(y3), np.array(y4), np.array(y5)


""" Loading data """
x_train, y_train_1, y_train_2, y_train_3, y_train_4, y_train_5 = read_data("./data_indices/train_indice.csv", augmentation=False) #augmentation=True, flip_peak=True, flip_time=True, add_noise=True)
x_test, y_test_1, y_test_2, y_test_3, y_test_4, y_test_5 = read_data("./data_indices/test_indice.csv")
print("Finished reading data!")


start imports
end imports
passed here
Reading data...
200 entries read...


<ipython-input-3-15a4b586a726>:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  datamat = np.arange(row, dtype=np.float)


400 entries read...
600 entries read...
800 entries read...
1000 entries read...
1200 entries read...
1400 entries read...
1600 entries read...
1800 entries read...
2000 entries read...
2200 entries read...
2400 entries read...
2600 entries read...
2800 entries read...
3000 entries read...
3200 entries read...
3400 entries read...
3600 entries read...
3800 entries read...
4000 entries read...
4200 entries read...
4400 entries read...
4600 entries read...
4800 entries read...
5000 entries read...
5200 entries read...
5400 entries read...
5600 entries read...
5800 entries read...
6000 entries read...
6200 entries read...
6400 entries read...
6600 entries read...
6800 entries read...
7000 entries read...
7200 entries read...
7400 entries read...
7600 entries read...
7800 entries read...
8000 entries read...
8200 entries read...
8400 entries read...
8600 entries read...
8800 entries read...
9000 entries read...
9200 entries read...
9400 entries read...
9600 entries read...
9800 entries rea

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, _, y_train_1, _, = train_test_split(x_train, y_train_1, train_size=0.4)

In [ ]:
# Trying to improve multitask!!!

#best result --> no augmentation

""" Functions for creating, compiling and training model"""

# Architecture:
# Main branch
# Branch1 => Decides wheter life threatening (VF or VT style) or not (SR and others)
# Branch2 => Classify the input


# labels,Rhythm
# AFb,Atrial Fibrillation
# AFt,Atrial Flutter
# SR,Sinus Rhythm
# SVT,Supraventricular Tachycardia
# VFb,Ventricular Fibrillation
# VFt,Ventricular Flutter
# VPD,Ventricular Premature Depolarizations
# VT,Ventricular Tachycardia


def gen_model():
    inputs = keras.layers.Input(shape=(1250, 1), name='input')

    main_branch = keras.layers.Conv1D(filters=3, kernel_size=6, strides=2, activation="relu") (inputs)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=5, kernel_size=5, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=10, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=32, kernel_size=1, strides=32, activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Flatten() (main_branch)
    main_branch = keras.layers.Dropout(0.5) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)

    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dropout(0.2) (branch1)

    # Decides wheter life threatening (VF or VT style) or not (SR and others)
    branch1 = keras.layers.Dense(2, activation="softmax", name='task_1_output') (main_branch)

    #branch2 = keras.layers.Dense(128, activation="relu") (main_branch)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dropout(0.2) (branch2)

    # Classify the input
    branch2 = keras.layers.Dense(8, activation="softmax", name='task_2_output') (main_branch)

    # mean
    branch3 = keras.layers.Dense(1, activation="relu", name='task_3_output') (main_branch)

    ## std
    #branch4 = keras.layers.Dense(1, activation="relu", name='task_4_output') (main_branch)
#
    ## var
    #branch5 = keras.layers.Dense(1, activation="relu", name='task_5_output') (main_branch)

    model = tf.keras.Model(inputs = inputs, outputs = [branch1, branch2, branch3,]) #branch4, branch5])

    return model

tf.keras.utils.plot_model(gen_model(), to_file='model_diagram.png', show_layer_activations=True,)

# loss_weight -> [0, 1]
# the weight for the second task is calculated by '1 - loss_weight'
def compile_model(model, loss_weight1, loss_weight2, loss_weight3, loss_weight4, loss_weight5):
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
                  loss={'task_1_output': 'binary_crossentropy',
                        'task_2_output': 'categorical_crossentropy',
                        'task_3_output': 'mse',
                        #'task_4_output': 'mse',
                        #'task_5_output': 'mse'
                        },
                  loss_weights={'task_1_output': loss_weight1,
                                'task_2_output': loss_weight2,
                                'task_3_output': loss_weight3,
                                #'task_4_output': loss_weight4,
                                #'task_5_output': loss_weight5
                                },
                  metrics={'task_1_output' : ["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           'task_2_output' : ["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           'task_3_output' : "mse",
                           #'task_4_output' : "mse",
                           #'task_5_output' : "mse"
                           })
    return model

def fit_batch(gamma_values, epochs=30, batch_size=32, save_models=False, models_dir="./trained_models", verbose=0):

    history = list()
    trained_models = list()
    test_scores = list()

    start_epoch = 10
    swa = SWA(start_epoch=start_epoch,
          lr_schedule='cyclic',
          swa_lr=0.0001,
          swa_lr2=0.0005,
          swa_freq=5,
          batch_size=batch_size,
          verbose=1)

    if save_models and not os.path.isdir(models_dir):
      os.mkdir(models_dir)

    print('Starting training on batch of models for gamma values ', gamma_values, '\n\n')

    for i, gamma in enumerate(gamma_values):

        print('Training model for gamma equal to ', gamma)
        model = gen_model()
        model = compile_model(model, gamma[0], gamma[1], gamma[2], gamma[3], gamma[4])
        start = time.time()
        model_history = model.fit({'input': x_train},
                            {'task_1_output': y_train_1, 'task_2_output': y_train_2, 'task_3_output': y_train_3,
                             #'task_4_output': y_train_4, 'task_5_output': y_train_5
                             },
                            epochs=epochs, batch_size=batch_size, verbose=verbose) # callbacks=[swa],
        print(f'Training time: {time.time() - start}\n')
        history.append(model_history)
        trained_models.append(model)
        if save_models:
          model.save(os.path.join(models_dir, f"{i}th-model"))

        test_score = model.evaluate({'input': x_test}, {'task_1_output': y_test_1, 'task_2_output': y_test_2, 'task_3_output': y_test_3,
                                                        #'task_4_output': y_test_4, 'task_5_output': y_test_5
                                                        })
        for metric, result in zip(model.metrics_names, test_score):
          print(f"{metric}: {result}")

        test_scores.append(test_score)

    return history, trained_models, test_scores

def plot_multitask_accuracies(gammas, training_history):

    counter = 0

    for history in training_history:

        print(f'\nPlotting Accuracy/Precision/Recall vs Epochs for value of gamma number {gammas[counter]}\n')
        plt.plot(range(len(history.history['task_1_output_accuracy'])), history.history['task_1_output_accuracy'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_accuracy'])), history.history['task_2_output_accuracy'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_precision'])), history.history['task_1_output_precision'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_precision'])), history.history['task_2_output_precision'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_recall'])), history.history['task_1_output_recall'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_recall'])), history.history['task_2_output_recall'], c='b', label='Task 2')
        plt.xlabel('Epochs')
        plt.legend()
        plt.show()
        counter += 1

"""Actually training the model"""
#gamma_values = [ [0.2, 0.2, 0.2, 0.2, 0.2],
#                [0.8, 0.1, 0.0333, 0.0333, 0.0333],
#                [0.5, 0.125, 0.125, 0.125, 0.125],
#                [0.5, 0.2, 0.1, 0.1, 0.1],
#                [0.6, 0.3, 0.033, 0.033, 0.033]]

gamma_values = [  [0.5, 0.25, 0.25, 0, 0],
                  [0.6, 0.2, 0.2, 0, 0],
                  [0.7, 0.2, 0.1, 0, 0],
                  [0.8, 0.1, 0.1, 0, 0],]

history, trained_models, _ = fit_batch(gamma_values, epochs=20, save_models=True, models_dir="./trained_models_sixth_augmentation", verbose=1)


In [ ]:
# Trying to improve multitask!!!

#best result --> no augmentation

""" Functions for creating, compiling and training model"""

# Architecture:
# Main branch
# Branch1 => Decides wheter life threatening (VF or VT style) or not (SR and others)
# Branch2 => Classify the input


# labels,Rhythm
# AFb,Atrial Fibrillation
# AFt,Atrial Flutter
# SR,Sinus Rhythm
# SVT,Supraventricular Tachycardia
# VFb,Ventricular Fibrillation
# VFt,Ventricular Flutter
# VPD,Ventricular Premature Depolarizations
# VT,Ventricular Tachycardia


def gen_model():
    inputs = keras.layers.Input(shape=(1250, 1), name='input')

    main_branch = keras.layers.Conv1D(filters=3, kernel_size=6, strides=2, activation="relu") (inputs)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=5, kernel_size=5, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=10, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=32, kernel_size=1, strides=32, activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    main_branch = keras.layers.Flatten() (main_branch)
    main_branch = keras.layers.Dropout(0.5) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    main_branch = keras.layers.BatchNormalization() (main_branch)

    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dropout(0.2) (branch1)

    # Decides wheter life threatening (VF or VT style) or not (SR and others)
    branch1 = keras.layers.Dense(2, activation="softmax", name='task_1_output') (main_branch)

    #branch2 = keras.layers.Dense(128, activation="relu") (main_branch)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dropout(0.2) (branch2)

    # Classify the input
    branch2 = keras.layers.Dense(8, activation="softmax", name='task_2_output') (main_branch)

    # mean
    branch3 = keras.layers.Dense(1, activation="relu", name='task_3_output') (main_branch)

    ## std
    #branch4 = keras.layers.Dense(1, activation="relu", name='task_4_output') (main_branch)
#
    ## var
    #branch5 = keras.layers.Dense(1, activation="relu", name='task_5_output') (main_branch)

    model = tf.keras.Model(inputs = inputs, outputs = [branch1, branch2, branch3,]) #branch4, branch5])

    return model

# loss_weight -> [0, 1]
# the weight for the second task is calculated by '1 - loss_weight'
def compile_model(model, loss_weight1, loss_weight2, loss_weight3, loss_weight4, loss_weight5):
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
                  loss={'task_1_output': 'binary_crossentropy',
                        'task_2_output': 'categorical_crossentropy',
                        'task_3_output': 'mse',
                        #'task_4_output': 'mse',
                        #'task_5_output': 'mse'
                        },
                  loss_weights={'task_1_output': loss_weight1,
                                'task_2_output': loss_weight2,
                                'task_3_output': loss_weight3,
                                #'task_4_output': loss_weight4,
                                #'task_5_output': loss_weight5
                                },
                  metrics={'task_1_output' : ["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           'task_2_output' : ["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           'task_3_output' : "mse",
                           #'task_4_output' : "mse",
                           #'task_5_output' : "mse"
                           })
    return model

def fit_batch(gamma_values, epochs=30, batch_size=32, save_models=False, models_dir="./trained_models", verbose=0):

    history = list()
    trained_models = list()
    test_scores = list()

    start_epoch = 10
    swa = SWA(start_epoch=start_epoch,
          lr_schedule='cyclic',
          swa_lr=0.0001,
          swa_lr2=0.0005,
          swa_freq=5,
          batch_size=batch_size,
          verbose=1)

    if save_models and not os.path.isdir(models_dir):
      os.mkdir(models_dir)

    print('Starting training on batch of models for gamma values ', gamma_values, '\n\n')

    for i, gamma in enumerate(gamma_values):

        print('Training model for gamma equal to ', gamma)
        model = gen_model()
        model = compile_model(model, gamma[0], gamma[1], gamma[2], gamma[3], gamma[4])
        start = time.time()
        model_history = model.fit({'input': x_train},
                            {'task_1_output': y_train_1, 'task_2_output': y_train_2, 'task_3_output': y_train_3,
                             #'task_4_output': y_train_4, 'task_5_output': y_train_5
                             },
                            epochs=epochs, batch_size=batch_size, verbose=verbose) # callbacks=[swa],
        print(f'Training time: {time.time() - start}\n')
        history.append(model_history)
        trained_models.append(model)
        if save_models:
          model.save(os.path.join(models_dir, f"{i}th-model"))

        test_score = model.evaluate({'input': x_test}, {'task_1_output': y_test_1, 'task_2_output': y_test_2, 'task_3_output': y_test_3,
                                                        #'task_4_output': y_test_4, 'task_5_output': y_test_5
                                                        })
        for metric, result in zip(model.metrics_names, test_score):
          print(f"{metric}: {result}")

        test_scores.append(test_score)

    return history, trained_models, test_scores

def plot_multitask_accuracies(gammas, training_history):

    counter = 0

    for history in training_history:

        print(f'\nPlotting Accuracy/Precision/Recall vs Epochs for value of gamma number {gammas[counter]}\n')
        plt.plot(range(len(history.history['task_1_output_accuracy'])), history.history['task_1_output_accuracy'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_accuracy'])), history.history['task_2_output_accuracy'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_precision'])), history.history['task_1_output_precision'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_precision'])), history.history['task_2_output_precision'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_recall'])), history.history['task_1_output_recall'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_recall'])), history.history['task_2_output_recall'], c='b', label='Task 2')
        plt.xlabel('Epochs')
        plt.legend()
        plt.show()
        counter += 1

"""Actually training the model"""
#gamma_values = [ [0.2, 0.2, 0.2, 0.2, 0.2],
#                [0.8, 0.1, 0.0333, 0.0333, 0.0333],
#                [0.5, 0.125, 0.125, 0.125, 0.125],
#                [0.5, 0.2, 0.1, 0.1, 0.1],
#                [0.6, 0.3, 0.033, 0.033, 0.033]]

gamma_values = [ [0.8, 0.2, 0, 0, 0],]

for _ in range(20):
  history, trained_models, _ = fit_batch(gamma_values, epochs=20, save_models=True, models_dir="./trained_models_sixth_augmentation", verbose=0)


Starting training on batch of models for gamma values  [[0.8, 0.2, 0, 0, 0]] 


Training model for gamma equal to  [0.8, 0.2, 0, 0, 0]
Training time: 183.06656885147095



176/176 [==============================] - 2s 5ms/step - loss: 0.4407 - task_1_output_loss: 0.3216 - task_2_output_loss: 0.9170 - task_3_output_loss: 0.8154 - task_1_output_accuracy: 0.9573 - task_1_output_precision_88: 0.9573 - task_1_output_recall_88: 0.9573 - task_2_output_accuracy: 0.8341 - task_2_output_precision_89: 0.8387 - task_2_output_recall_89: 0.8302 - task_3_output_mse: 0.8154
loss: 0.44065260887145996
task_1_output_loss: 0.3215768337249756
task_2_output_loss: 0.916955828666687
task_3_output_loss: 0.8154197335243225
task_1_output_accuracy: 0.9573333263397217
task_1_output_precision_88: 0.9573333263397217
task_1_output_recall_88: 0.9573333263397217
task_2_output_accuracy: 0.8341333270072937
task_2_output_precision_89: 0.8387212753295898
task_2_output_recall_89: 0.8302222490310669
task_3_output_mse: 0.8154197335243225
Starting training on batch of models for gamma values  [[0.8, 0.2, 0, 0, 0]] 


Training model for gamma equal to  [0.8, 0.2, 0, 0, 0]
Training time: 185.30848

176/176 [==============================] - 2s 5ms/step - loss: 0.3907 - task_1_output_loss: 0.2793 - task_2_output_loss: 0.8362 - task_3_output_loss: 1.0999 - task_1_output_accuracy: 0.9582 - task_1_output_precision_90: 0.9582 - task_1_output_recall_90: 0.9582 - task_2_output_accuracy: 0.8231 - task_2_output_precision_91: 0.8285 - task_2_output_recall_91: 0.8169 - task_3_output_mse: 1.0999
loss: 0.39068564772605896
task_1_output_loss: 0.2793075740337372
task_2_output_loss: 0.836198091506958
task_3_output_loss: 1.0999072790145874
task_1_output_accuracy: 0.9582222104072571
task_1_output_precision_90: 0.9582222104072571
task_1_output_recall_90: 0.9582222104072571
task_2_output_accuracy: 0.8231111168861389
task_2_output_precision_91: 0.8285250663757324
task_2_output_recall_91: 0.8168888688087463
task_3_output_mse: 1.0999072790145874
Starting training on batch of models for gamma values  [[0.8, 0.2, 0, 0, 0]] 


Training model for gamma equal to  [0.8, 0.2, 0, 0, 0]
Training time: 206.38521

176/176 [==============================] - 2s 6ms/step - loss: 0.4333 - task_1_output_loss: 0.3262 - task_2_output_loss: 0.8614 - task_3_output_loss: 0.3196 - task_1_output_accuracy: 0.9532 - task_1_output_precision_92: 0.9532 - task_1_output_recall_92: 0.9532 - task_2_output_accuracy: 0.8263 - task_2_output_precision_93: 0.8379 - task_2_output_recall_93: 0.8105 - task_3_output_mse: 0.3196
loss: 0.4332614243030548
task_1_output_loss: 0.3262348473072052
task_2_output_loss: 0.8613669872283936
task_3_output_loss: 0.31957098841667175
task_1_output_accuracy: 0.9532444477081299
task_1_output_precision_92: 0.9532444477081299
task_1_output_recall_92: 0.9532444477081299
task_2_output_accuracy: 0.8263111114501953
task_2_output_precision_93: 0.8378974199295044
task_2_output_recall_93: 0.8104888796806335
task_3_output_mse: 0.31957098841667175
Starting training on batch of models for gamma values  [[0.8, 0.2, 0, 0, 0]] 


Training model for gamma equal to  [0.8, 0.2, 0, 0, 0]
Training time: 205.640

176/176 [==============================] - 2s 6ms/step - loss: 0.3708 - task_1_output_loss: 0.2620 - task_2_output_loss: 0.8061 - task_3_output_loss: 0.3874 - task_1_output_accuracy: 0.9589 - task_1_output_precision_94: 0.9589 - task_1_output_recall_94: 0.9589 - task_2_output_accuracy: 0.8334 - task_2_output_precision_95: 0.8402 - task_2_output_recall_95: 0.8251 - task_3_output_mse: 0.3874
loss: 0.370828241109848
task_1_output_loss: 0.2620190680027008
task_2_output_loss: 0.8060644865036011
task_3_output_loss: 0.3874266445636749
task_1_output_accuracy: 0.9589333534240723
task_1_output_precision_94: 0.9589333534240723
task_1_output_recall_94: 0.9589333534240723
task_2_output_accuracy: 0.8334222435951233
task_2_output_precision_95: 0.8401520848274231
task_2_output_recall_95: 0.8250666856765747
task_3_output_mse: 0.3874266445636749
Starting training on batch of models for gamma values  [[0.8, 0.2, 0, 0, 0]] 


Training model for gamma equal to  [0.8, 0.2, 0, 0, 0]
Training time: 206.124921

...

176/176 [==============================] - 2s 6ms/step - loss: 0.3823 - task_1_output_loss: 0.2723 - task_2_output_loss: 0.8223 - task_3_output_loss: 0.8219 - task_1_output_accuracy: 0.9529 - task_1_output_precision_118: 0.9529 - task_1_output_recall_118: 0.9529 - task_2_output_accuracy: 0.8212 - task_2_output_precision_119: 0.8304 - task_2_output_recall_119: 0.8137 - task_3_output_mse: 0.8219
loss: 0.38233041763305664
task_1_output_loss: 0.27234143018722534
task_2_output_loss: 0.8222869634628296
task_3_output_loss: 0.8219249844551086
task_1_output_accuracy: 0.9528889060020447
task_1_output_precision_118: 0.9528889060020447
task_1_output_recall_118: 0.9528889060020447
task_2_output_accuracy: 0.8211555480957031
task_2_output_precision_119: 0.8303701281547546
task_2_output_recall_119: 0.8136888742446899
task_3_output_mse: 0.8219249844551086
Starting training on batch of models for gamma values  [[0.8, 0.2, 0, 0, 0]] 


Training model for gamma equal to  [0.8, 0.2, 0, 0, 0]
Training time:

176/176 [==============================] - 2s 6ms/step - loss: 0.3684 - task_1_output_loss: 0.2587 - task_2_output_loss: 0.8070 - task_3_output_loss: 0.4890 - task_1_output_accuracy: 0.9403 - task_1_output_precision_120: 0.9403 - task_1_output_recall_120: 0.9403 - task_2_output_accuracy: 0.8114 - task_2_output_precision_121: 0.8240 - task_2_output_recall_121: 0.8005 - task_3_output_mse: 0.4890
loss: 0.3683888018131256
task_1_output_loss: 0.2587340176105499
task_2_output_loss: 0.8070075511932373
task_3_output_loss: 0.48900365829467773
task_1_output_accuracy: 0.9402666687965393
task_1_output_precision_120: 0.9402666687965393
task_1_output_recall_120: 0.9402666687965393
task_2_output_accuracy: 0.811377763748169
task_2_output_precision_121: 0.8239707350730896
task_2_output_recall_121: 0.8005333542823792
task_3_output_mse: 0.48900365829467773
Starting training on batch of models for gamma values  [[0.8, 0.2, 0, 0, 0]] 


Training model for gamma equal to  [0.8, 0.2, 0, 0, 0]
Training time: 

176/176 [==============================] - 2s 5ms/step - loss: 0.3577 - task_1_output_loss: 0.2341 - task_2_output_loss: 0.8520 - task_3_output_loss: 0.3969 - task_1_output_accuracy: 0.9520 - task_1_output_precision_122: 0.9520 - task_1_output_recall_122: 0.9520 - task_2_output_accuracy: 0.8160 - task_2_output_precision_123: 0.8214 - task_2_output_recall_123: 0.8103 - task_3_output_mse: 0.3969
loss: 0.3576972782611847
task_1_output_loss: 0.2341300994157791
task_2_output_loss: 0.8519657850265503
task_3_output_loss: 0.39688295125961304
task_1_output_accuracy: 0.9520000219345093
task_1_output_precision_122: 0.9520000219345093
task_1_output_recall_122: 0.9520000219345093
task_2_output_accuracy: 0.8159999847412109
task_2_output_precision_123: 0.821409285068512
task_2_output_recall_123: 0.8103111386299133
task_3_output_mse: 0.39688295125961304
Starting training on batch of models for gamma values  [[0.8, 0.2, 0, 0, 0]] 


Training model for gamma equal to  [0.8, 0.2, 0, 0, 0]
Training time: 

176/176 [==============================] - 2s 6ms/step - loss: 0.4208 - task_1_output_loss: 0.3050 - task_2_output_loss: 0.8841 - task_3_output_loss: 1.5189 - task_1_output_accuracy: 0.9465 - task_1_output_precision_124: 0.9465 - task_1_output_recall_124: 0.9465 - task_2_output_accuracy: 0.8114 - task_2_output_precision_125: 0.8177 - task_2_output_recall_125: 0.8028 - task_3_output_mse: 1.5189
loss: 0.4208431541919708
task_1_output_loss: 0.3050307631492615
task_2_output_loss: 0.8840928077697754
task_3_output_loss: 1.5189106464385986
task_1_output_accuracy: 0.9464889168739319
task_1_output_precision_124: 0.9464889168739319
task_1_output_recall_124: 0.9464889168739319
task_2_output_accuracy: 0.811377763748169
task_2_output_precision_125: 0.817671537399292
task_2_output_recall_125: 0.8028444647789001
task_3_output_mse: 1.5189106464385986
Starting training on batch of models for gamma values  [[0.8, 0.2, 0, 0, 0]] 


Training model for gamma equal to  [0.8, 0.2, 0, 0, 0]
Training time: 206

176/176 [==============================] - 2s 6ms/step - loss: 0.4275 - task_1_output_loss: 0.2946 - task_2_output_loss: 0.9594 - task_3_output_loss: 0.5969 - task_1_output_accuracy: 0.9479 - task_1_output_precision_126: 0.9479 - task_1_output_recall_126: 0.9479 - task_2_output_accuracy: 0.8153 - task_2_output_precision_127: 0.8207 - task_2_output_recall_127: 0.8098 - task_3_output_mse: 0.5969
loss: 0.42754852771759033
task_1_output_loss: 0.29458341002464294
task_2_output_loss: 0.9594088792800903
task_3_output_loss: 0.5968636274337769
task_1_output_accuracy: 0.9479110836982727
task_1_output_precision_126: 0.9479110836982727
task_1_output_recall_126: 0.9479110836982727
task_2_output_accuracy: 0.8152889013290405
task_2_output_precision_127: 0.8207207322120667
task_2_output_recall_127: 0.8097777962684631
task_3_output_mse: 0.5968636274337769


REDES RECORRENTES ...





In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], 1250, 1, 1))
x_test = np.reshape(x_test, (x_test.shape[0], 1250, 1, 1))

In [ ]:
x_train.shape

(24588, 1250)

In [ ]:
x_test.shape

(5625, 1250, 1, 3)

In [ ]:
x_train.shape = (24588, 1250, 1)
x_test.shape = (x_test.shape[0], 1250, 1)
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)

In [ ]:
# Trying to improve multitask!!!

#best result --> no augmentation

""" Functions for creating, compiling and training model"""

# Architecture:
# Main branch
# Branch1 => Decides wheter life threatening (VF or VT style) or not (SR and others)
# Branch2 => Classify the input


# labels,Rhythm
# AFb,Atrial Fibrillation
# AFt,Atrial Flutter
# SR,Sinus Rhythm
# SVT,Supraventricular Tachycardia
# VFb,Ventricular Fibrillation
# VFt,Ventricular Flutter
# VPD,Ventricular Premature Depolarizations
# VT,Ventricular Tachycardia


def gen_model():
    inputs = keras.layers.Input(shape=(1250, 1, 3), name='input')
    effNet = tf.keras.applications.EfficientNetB0(weights="imagenet", input_tensor=inputs, include_top=False)
    main_branch = keras.layers.Flatten()(effNet.output)
    main_branch = keras.layers.Reshape((40, 1280))(main_branch)
    main_branch = keras.layers.LSTM(units=10, return_sequences=True) (main_branch)
    main_branch = keras.layers.LSTM(units=10, return_sequences=True) (main_branch)
    main_branch = keras.layers.LSTM(units=10) (main_branch)
    main_branch = keras.layers.Dropout(0.2) (main_branch)


    #main_branch = keras.layers.Conv1D(filters=3, kernel_size=6, strides=2, activation="relu") (inputs)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=5, kernel_size=5, strides=2,  activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=10, kernel_size=4, strides=2,  activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=32, kernel_size=1, strides=32, activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Flatten() (main_branch)
    #main_branch = keras.layers.Dropout(0.5) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)

    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dropout(0.2) (branch1)

    # Decides wheter life threatening (VF or VT style) or not (SR and others)
    branch1 = keras.layers.Dense(2, activation="softmax", name='task_1_output') (main_branch)

    #branch2 = keras.layers.Dense(128, activation="relu") (main_branch)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dropout(0.2) (branch2)

    # Classify the input
    branch2 = keras.layers.Dense(8, activation="softmax", name='task_2_output') (main_branch)

    # mean
    branch3 = keras.layers.Dense(1, activation="relu", name='task_3_output') (main_branch)

    ## std
    #branch4 = keras.layers.Dense(1, activation="relu", name='task_4_output') (main_branch)
#
    ## var
    #branch5 = keras.layers.Dense(1, activation="relu", name='task_5_output') (main_branch)

    model = tf.keras.Model(inputs = inputs, outputs = [branch1, branch2, branch3,]) #branch4, branch5])

    return model

# loss_weight -> [0, 1]
# the weight for the second task is calculated by '1 - loss_weight'
def compile_model(model, loss_weight1, loss_weight2, loss_weight3, loss_weight4, loss_weight5):
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
                  loss={'task_1_output': 'binary_crossentropy',
                        'task_2_output': 'categorical_crossentropy',
                        'task_3_output': 'mse',
                        #'task_4_output': 'mse',
                        #'task_5_output': 'mse'
                        },
                  loss_weights={'task_1_output': loss_weight1,
                                'task_2_output': loss_weight2,
                                'task_3_output': loss_weight3,
                                #'task_4_output': loss_weight4,
                                #'task_5_output': loss_weight5
                                },
                  metrics={'task_1_output' : ["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           'task_2_output' : ["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           'task_3_output' : "mse",
                           #'task_4_output' : "mse",
                           #'task_5_output' : "mse"
                           })
    return model

def fit_batch(gamma_values, epochs=30, batch_size=32, save_models=False, models_dir="./trained_models", verbose=0):

    history = list()
    trained_models = list()
    test_scores = list()

    start_epoch = 10
    swa = SWA(start_epoch=start_epoch,
          lr_schedule='cyclic',
          swa_lr=0.0001,
          swa_lr2=0.0005,
          swa_freq=5,
          batch_size=batch_size,
          verbose=1)

    if save_models and not os.path.isdir(models_dir):
      os.mkdir(models_dir)

    print('Starting training on batch of models for gamma values ', gamma_values, '\n\n')

    for i, gamma in enumerate(gamma_values):

        print('Training model for gamma equal to ', gamma)
        model = gen_model()
        model = compile_model(model, gamma[0], gamma[1], gamma[2], gamma[3], gamma[4])
        start = time.time()
        model_history = model.fit({'input': x_train},
                            {'task_1_output': y_train_1, 'task_2_output': y_train_2, 'task_3_output': y_train_3,
                             #'task_4_output': y_train_4, 'task_5_output': y_train_5
                             },
                            epochs=epochs, batch_size=batch_size, verbose=verbose) # callbacks=[swa],
        print(f'Training time: {time.time() - start}\n')
        history.append(model_history)
        trained_models.append(model)
        if save_models:
          model.save(os.path.join(models_dir, f"{i}th-model"))

        test_score = model.evaluate({'input': x_test}, {'task_1_output': y_test_1, 'task_2_output': y_test_2, 'task_3_output': y_test_3,
                                                        #'task_4_output': y_test_4, 'task_5_output': y_test_5
                                                        })
        for metric, result in zip(model.metrics_names, test_score):
          print(f"{metric}: {result}")

        test_scores.append(test_score)

    return history, trained_models, test_scores

def plot_multitask_accuracies(gammas, training_history):

    counter = 0

    for history in training_history:

        print(f'\nPlotting Accuracy/Precision/Recall vs Epochs for value of gamma number {gammas[counter]}\n')
        plt.plot(range(len(history.history['task_1_output_accuracy'])), history.history['task_1_output_accuracy'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_accuracy'])), history.history['task_2_output_accuracy'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_precision'])), history.history['task_1_output_precision'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_precision'])), history.history['task_2_output_precision'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_recall'])), history.history['task_1_output_recall'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_recall'])), history.history['task_2_output_recall'], c='b', label='Task 2')
        plt.xlabel('Epochs')
        plt.legend()
        plt.show()
        counter += 1

"""Actually training the model"""
#gamma_values = [ [0.2, 0.2, 0.2, 0.2, 0.2],
#                [0.8, 0.1, 0.0333, 0.0333, 0.0333],
#                [0.5, 0.125, 0.125, 0.125, 0.125],
#                [0.5, 0.2, 0.1, 0.1, 0.1],
#                [0.6, 0.3, 0.033, 0.033, 0.033]]

gamma_values = [  #[0.5, 0.25, 0.25, 0, 0],
                  #[0.6, 0.2, 0.2, 0, 0],
                  [0.7, 0.2, 0.1, 0, 0],
                  #[0.8, 0.1, 0.1, 0, 0],
                  ]

history, trained_models, _ = fit_batch(gamma_values, epochs=50, save_models=True, models_dir="./trained_models_sixth_augmentation", verbose=1)


Starting training on batch of models for gamma values  [[0.7, 0.2, 0.1, 0, 0]] 


Training model for gamma equal to  [0.7, 0.2, 0.1, 0, 0]
Epoch 1/50
769/769 [==============================] - 124s 78ms/step - loss: 0.4667 - task_1_output_loss: 0.2881 - task_2_output_loss: 1.3107 - task_3_output_loss: 0.0290 - task_1_output_accuracy: 0.9522 - task_1_output_precision_2: 0.9522 - task_1_output_recall_2: 0.9522 - task_2_output_accuracy: 0.6701 - task_2_output_precision_3: 0.8614 - task_2_output_recall_3: 0.2370 - task_3_output_mse: 0.0290
Epoch 2/50
769/769 [==============================] - 58s 76ms/step - loss: 0.2185 - task_1_output_loss: 0.1018 - task_2_output_loss: 0.7301 - task_3_output_loss: 0.0121 - task_1_output_accuracy: 0.9836 - task_1_output_precision_2: 0.9836 - task_1_output_recall_2: 0.9836 - task_2_output_accuracy: 0.8457 - task_2_output_precision_3: 0.8629 - task_2_output_recall_3: 0.7632 - task_3_output_mse: 0.0121
Epoch 3/50
769/769 [==============================] - 58

TypeError: ignored

In [ ]:
# Trying to improve multitask!!!

#best result --> no augmentation

""" Functions for creating, compiling and training model"""

# Architecture:
# Main branch
# Branch1 => Decides wheter life threatening (VF or VT style) or not (SR and others)
# Branch2 => Classify the input


# labels,Rhythm
# AFb,Atrial Fibrillation
# AFt,Atrial Flutter
# SR,Sinus Rhythm
# SVT,Supraventricular Tachycardia
# VFb,Ventricular Fibrillation
# VFt,Ventricular Flutter
# VPD,Ventricular Premature Depolarizations
# VT,Ventricular Tachycardia


def gen_model():
    inputs = keras.layers.Input(shape=(1250, 1, 3), name='input')
    effNet = tf.keras.applications.EfficientNetB0(weights="imagenet", input_tensor=inputs, include_top=False)
    main_branch = keras.layers.Flatten()(effNet.output)
    main_branch = keras.layers.Reshape((40, 1280))(main_branch)
    main_branch = keras.layers.LSTM(units=10, return_sequences=True) (main_branch)
    main_branch = keras.layers.LSTM(units=10, return_sequences=True) (main_branch)
    main_branch = keras.layers.LSTM(units=10) (main_branch)
    main_branch = keras.layers.Dropout(0.2) (main_branch)


    #main_branch = keras.layers.Conv1D(filters=3, kernel_size=6, strides=2, activation="relu") (inputs)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=5, kernel_size=5, strides=2,  activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=10, kernel_size=4, strides=2,  activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,  activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Conv1D(filters=32, kernel_size=1, strides=32, activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)
    #main_branch = keras.layers.Flatten() (main_branch)
    #main_branch = keras.layers.Dropout(0.5) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.Dropout(0.1) (main_branch)
    #main_branch = keras.layers.Dense(512, activation="relu") (main_branch)
    #main_branch = keras.layers.BatchNormalization() (main_branch)

    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dense(128, activation="relu") (branch1)
    #branch1 = keras.layers.BatchNormalization() (branch1)
    #branch1 = keras.layers.Dropout(0.2) (branch1)

    # Decides wheter life threatening (VF or VT style) or not (SR and others)
    branch1 = keras.layers.Dense(2, activation="softmax", name='task_1_output') (main_branch)

    #branch2 = keras.layers.Dense(128, activation="relu") (main_branch)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dense(128, activation="relu") (branch2)
    #branch2 = keras.layers.BatchNormalization() (branch2)
    #branch2 = keras.layers.Dropout(0.2) (branch2)

    # Classify the input
    branch2 = keras.layers.Dense(8, activation="softmax", name='task_2_output') (main_branch)

    # mean
    branch3 = keras.layers.Dense(1, activation="relu", name='task_3_output') (main_branch)

    ## std
    #branch4 = keras.layers.Dense(1, activation="relu", name='task_4_output') (main_branch)
#
    ## var
    #branch5 = keras.layers.Dense(1, activation="relu", name='task_5_output') (main_branch)

    model = tf.keras.Model(inputs = inputs, outputs = [branch1, branch2, branch3,]) #branch4, branch5])

    return model

# loss_weight -> [0, 1]
# the weight for the second task is calculated by '1 - loss_weight'
def compile_model(model, loss_weight1, loss_weight2, loss_weight3, loss_weight4, loss_weight5):
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
                  loss={'task_1_output': 'binary_crossentropy',
                        'task_2_output': 'categorical_crossentropy',
                        'task_3_output': 'mse',
                        #'task_4_output': 'mse',
                        #'task_5_output': 'mse'
                        },
                  loss_weights={'task_1_output': loss_weight1,
                                'task_2_output': loss_weight2,
                                'task_3_output': loss_weight3,
                                #'task_4_output': loss_weight4,
                                #'task_5_output': loss_weight5
                                },
                  metrics={'task_1_output' : ["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           'task_2_output' : ["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                           'task_3_output' : "mse",
                           #'task_4_output' : "mse",
                           #'task_5_output' : "mse"
                           })
    return model

def fit_batch(gamma_values, epochs=30, batch_size=32, save_models=False, models_dir="./trained_models", verbose=0):

    history = list()
    trained_models = list()
    test_scores = list()

    start_epoch = 10
    swa = SWA(start_epoch=start_epoch,
          lr_schedule='cyclic',
          swa_lr=0.0001,
          swa_lr2=0.0005,
          swa_freq=5,
          batch_size=batch_size,
          verbose=1)

    if save_models and not os.path.isdir(models_dir):
      os.mkdir(models_dir)

    print('Starting training on batch of models for gamma values ', gamma_values, '\n\n')

    for i, gamma in enumerate(gamma_values):

        print('Training model for gamma equal to ', gamma)
        model = gen_model()
        model = compile_model(model, gamma[0], gamma[1], gamma[2], gamma[3], gamma[4])
        start = time.time()
        model_history = model.fit({'input': x_train},
                            {'task_1_output': y_train_1, 'task_2_output': y_train_2, 'task_3_output': y_train_3,
                             #'task_4_output': y_train_4, 'task_5_output': y_train_5
                             },
                            epochs=epochs, batch_size=batch_size, verbose=verbose) # callbacks=[swa],
        print(f'Training time: {time.time() - start}\n')
        history.append(model_history)
        trained_models.append(model)
        #if save_models:
        #  model.save(os.path.join(models_dir, f"{i}th-model"))

        test_score = model.evaluate({'input': x_test}, {'task_1_output': y_test_1, 'task_2_output': y_test_2, 'task_3_output': y_test_3,
                                                        #'task_4_output': y_test_4, 'task_5_output': y_test_5
                                                        })
        for metric, result in zip(model.metrics_names, test_score):
          print(f"{metric}: {result}")

        test_scores.append(test_score)

    return history, trained_models, test_scores

def plot_multitask_accuracies(gammas, training_history):

    counter = 0

    for history in training_history:

        print(f'\nPlotting Accuracy/Precision/Recall vs Epochs for value of gamma number {gammas[counter]}\n')
        plt.plot(range(len(history.history['task_1_output_accuracy'])), history.history['task_1_output_accuracy'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_accuracy'])), history.history['task_2_output_accuracy'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_precision'])), history.history['task_1_output_precision'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_precision'])), history.history['task_2_output_precision'], c='b', label='Task 2')
        plt.plot(range(len(history.history['task_1_output_recall'])), history.history['task_1_output_recall'], c='r', label='Task 1')
        plt.plot(range(len(history.history['task_2_output_recall'])), history.history['task_2_output_recall'], c='b', label='Task 2')
        plt.xlabel('Epochs')
        plt.legend()
        plt.show()
        counter += 1

"""Actually training the model"""
#gamma_values = [ [0.2, 0.2, 0.2, 0.2, 0.2],
#                [0.8, 0.1, 0.0333, 0.0333, 0.0333],
#                [0.5, 0.125, 0.125, 0.125, 0.125],
#                [0.5, 0.2, 0.1, 0.1, 0.1],
#                [0.6, 0.3, 0.033, 0.033, 0.033]]

gamma_values = [  #[0.5, 0.25, 0.25, 0, 0],
                  #[0.6, 0.2, 0.2, 0, 0],
                  [0.7, 0.2, 0.1, 0, 0],
                  #[0.8, 0.1, 0.1, 0, 0],
                  ]

history, trained_models, _ = fit_batch(gamma_values, epochs=50, save_models=True, models_dir="./trained_models_sixth_augmentation", verbose=1)


Starting training on batch of models for gamma values  [[0.7, 0.2, 0.1, 0, 0]] 


Training model for gamma equal to  [0.7, 0.2, 0.1, 0, 0]
Epoch 1/50
769/769 [==============================] - 108s 76ms/step - loss: 0.4549 - task_1_output_loss: 0.3216 - task_2_output_loss: 1.1296 - task_3_output_loss: 0.0387 - task_1_output_accuracy: 0.9526 - task_1_output_precision_6: 0.9526 - task_1_output_recall_6: 0.9526 - task_2_output_accuracy: 0.8014 - task_2_output_precision_7: 0.8613 - task_2_output_recall_7: 0.3931 - task_3_output_mse: 0.0387
Epoch 2/50
769/769 [==============================] - 58s 75ms/step - loss: 0.2129 - task_1_output_loss: 0.1170 - task_2_output_loss: 0.6404 - task_3_output_loss: 0.0291 - task_1_output_accuracy: 0.9866 - task_1_output_precision_6: 0.9866 - task_1_output_recall_6: 0.9866 - task_2_output_accuracy: 0.8406 - task_2_output_precision_7: 0.8823 - task_2_output_recall_7: 0.7978 - task_3_output_mse: 0.0291
Epoch 3/50
769/769 [==============================] - 59